# Exploratory analysis
## Using simulated data

In [1]:
import sgkit as sg
import xarray as xr
import numpy as np

In [5]:
ds = sg.simulate_genotype_call_dataset(
    n_variant=1000, n_sample=15, n_contig=23, missing_pct=1
)

In [6]:
sg.display_genotypes(sim_data, max_variants=8, max_samples=8)

samples,S0,S1,S2,S3,...,S246,S247,S248,S249
variants,,,,,,,,,
0,0/0,1/0,1/0,0/1,...,1/1,0/0,1/1,1/0
1,0/0,0/1,1/1,1/1,...,0/1,1/0,0/.,0/0
2,1/1,0/1,1/0,0/1,...,0/0,1/1,0/1,./1
3,1/0,1/0,1/0,1/0,...,1/1,1/0,0/1,1/.
...,...,...,...,...,...,...,...,...,...
39,0/.,1/1,1/1,0/0,...,./.,0/0,1/0,./0
40,0/0,./0,1/0,1/1,...,1/.,1/0,1/1,./1
41,0/1,0/1,1/1,1/0,...,0/0,0/0,0/1,./1
42,1/0,0/1,1/0,1/1,...,0/1,1/0,0/0,0/1


### Cohorts
Cohorts are groups of samples (think population). They are defined by mapping a cohort index onto samples

In [12]:
sim_data_cohort = sg.simulate_genotype_call_dataset(n_variant=100, n_sample=10)
sim_data_cohort["sample_cohort"] = xr.DataArray(np.array([-1, 0, 1, 1, 1, 1, 0, 2, 2, 2]), dims="samples")
sim_data_cohort

<xarray.Dataset> Size: 6kB
Dimensions:             (contigs: 1, variants: 100, alleles: 2, samples: 10,
                         ploidy: 2)
Dimensions without coordinates: contigs, variants, alleles, samples, ploidy
Data variables:
    contig_id           (contigs) <U1 4B '0'
    variant_contig      (variants) int64 800B 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
    variant_position    (variants) int64 800B 0 1 2 3 4 5 ... 94 95 96 97 98 99
    variant_allele      (variants, alleles) |S1 200B b'T' b'T' ... b'T' b'G'
    sample_id           (samples) <U2 80B 'S0' 'S1' 'S2' 'S3' ... 'S7' 'S8' 'S9'
    call_genotype       (variants, samples, ploidy) int8 2kB 0 0 1 0 ... 0 1 1 1
    call_genotype_mask  (variants, samples, ploidy) bool 2kB False ... False
    sample_cohort       (samples) int64 80B -1 0 1 1 1 1 0 2 2 2
Attributes:
    contigs:  ['0']
    source:   sgkit-0.10.0

Using `sgkit.Fst()` produces stats for pairs (think pairwise FST by population).

In [ ]:
sim_data_cohort = sg.window_by_variant(sim_data_cohort, size=20)

sim_data_cohort = sg.Fst(sim_data_cohort)

cohort_names = ["Africa", "Asia", "Europe"]

sim_data_cohort = sim_data_cohort.assign_coords({"cohorts_0": cohort_names, "cohorts_1": cohort_names})

sim_data_cohort.stat_Fst.sel(cohorts_0="Africa", cohorts_1="Asia").values

array([ 0.00823045, -0.02510823,  0.04372041,  0.12240725, -0.11305475])

## Using my test data
`test/ProjTaxaTest.vcf`

In [2]:
# Load data into dataset object
ds = sg.load_dataset("../test/ProjTaxaFilt.vcz")

In [3]:
# Show data
print(ds.sizes["variants"])
# sg.display_genotypes(ds, max_variants=8, max_samples=8)
ds.call_genotype

35


<xarray.DataArray 'call_genotype' (variants: 35, samples: 15, ploidy: 2)> Size: 1kB
dask.array<open_dataset-call_genotype, shape=(35, 15, 2), dtype=int8, chunksize=(35, 15, 2), chunktype=numpy.ndarray>
Dimensions without coordinates: variants, samples, ploidy
Attributes:
    description:

### FST

In [4]:
ds.sample_id.values

array(['8N05240',
       '8N05890',
       '8N06612',
       '8N73248',
       '8N73604',
       'K006',
       'K010',
       'K011',
       'K015',
       'K019',
       'Lesina_280',
       'Lesina_281',
       'Lesina_282',
       'Lesina_285',
       'Lesina_286'],
      dtype=object)

In [ ]:
# ds["sample_cohort"] = xr.DataArray(np.array([0] * 5 + [1] * 5 + [2] * 5), dims="samples")
# cohort_names = ["8N", "K0", "Lesina"]
# ds = ds.assign_coords({"cohorts_0": cohort_names, "cohorts_1": cohort_names})

In [78]:
sample_cohort = np.array([0] * 5 + [1] * 5 + [2] * 5)
ds["sample_cohort"] = xr.DataArray(sample_cohort, dims="samples")
sg.window_by_variant(ds, size=10)
# sg.divergence(ds)

<xarray.Dataset> Size: 81kB
Dimensions:             (windows: 115, contigs: 23, variants: 1000, alleles: 2,
                         samples: 15, ploidy: 2)
Dimensions without coordinates: windows, contigs, variants, alleles, samples,
                                ploidy
Data variables:
    window_contig       (windows) int64 920B 0 0 0 0 0 1 1 ... 21 22 22 22 22 22
    window_start        (windows) int64 920B 0 10 20 30 40 ... 967 977 987 997
    window_stop         (windows) int64 920B 10 20 30 40 44 ... 977 987 997 1000
    contig_id           (contigs) <U2 184B '0' '1' '2' '3' ... '20' '21' '22'
    variant_contig      (variants) int64 8kB 0 0 0 0 0 0 0 ... 22 22 22 22 22 22
    variant_position    (variants) int64 8kB 0 1 2 3 4 5 6 ... 37 38 39 40 41 42
    variant_allele      (variants, alleles) |S1 2kB b'A' b'T' b'A' ... b'C' b'T'
    sample_id           (samples) <U3 180B 'S0' 'S1' 'S2' ... 'S12' 'S13' 'S14'
    call_genotype       (variants, samples, ploidy) int8 30kB 0 0 1 0 ... 0 0 0
    call_genotype_mask  (variants, samples, ploidy) bool 30kB False ... False
    sample_cohort       (samples) int64 120B 0 0 0 0 0 1 1 1 1 1 2 2 2 2 2
Attributes:
    contigs:  ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', ...
    source:   sgkit-0.10.0

In [79]:
fst = sg.Fst(ds)["stat_Fst"].values
len(fst)

1000

In [91]:
import sgkit as sg
import numpy as np

# 1) Load dataset (replace with your path)

# 2) Assign cohorts (example: first half = popA, second half = popB)
# n = ds.dims["samples"]
# ds = ds.assign_coords(
#     sample_population=("samples", np.where(np.arange(n) < n // 2, "popA", "popB"))
# )
sample_cohort = np.array([0] * 5 + [1] * 5 + [2] * 5)
ds["sample_cohort"] = xr.DataArray(sample_cohort, dims="samples")

# 3) Window the variants (e.g., 1000 variants per window)
ds_win = sg.window_by_variant(
    ds, size=1000
)

# 4) Compute FST per window
fst_ds = sg.Fst(
    ds_win#,
    # group="sample_population",
    # call_genotype="call_genotype",
    # window="window_id"
)

# 5) View FST results
print(fst_ds.values)


<bound method Mapping.values of <xarray.Dataset> Size: 160kB
Dimensions:              (windows: 23, cohorts_0: 3, cohorts_1: 3,
                          variants: 1000, cohorts: 3, alleles: 2, samples: 15,
                          contigs: 23, ploidy: 2)
Dimensions without coordinates: windows, cohorts_0, cohorts_1, variants,
                                cohorts, alleles, samples, contigs, ploidy
Data variables: (12/15)
    stat_Fst             (windows, cohorts_0, cohorts_1) float64 2kB dask.array<chunksize=(23, 3, 3), meta=np.ndarray>
    stat_divergence      (windows, cohorts_0, cohorts_1) float64 2kB dask.array<chunksize=(23, 3, 3), meta=np.ndarray>
    cohort_allele_count  (variants, cohorts, alleles) uint64 48kB dask.array<chunksize=(1000, 3, 2), meta=np.ndarray>
    call_allele_count    (variants, samples, alleles) uint8 30kB dask.array<chunksize=(1000, 15, 2), meta=np.ndarray>
    window_contig        (windows) int64 184B 0 1 2 3 4 5 ... 17 18 19 20 21 22
    window_start 